In [1]:
from __future__ import print_function

import argparse
import numpy as np

import neuroglancer
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from glob import glob

In [2]:
stack = 'MD594'
source_dir = os.environ['ROOT_DIR']
fp = os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
with open(source_dir+fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}
fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
annotation = np.load(source_dir+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')
contours_struc = contours.groupby('name')
valid_sections = np.sort(contours['section'].unique())

In [3]:
contours[(contours['name']=='VCA') & (contours['section']==94)]['vertices'].to_list()[0]

array([[7846.35119146, 7025.37862291],
       [7727.78491719, 6973.16629019],
       [7662.61929633, 6922.79994021],
       [7665.50842641, 6848.03215937],
       [7804.54138864, 6725.98528806],
       [7876.96147159, 6692.33823903],
       [8004.30597918, 6660.50284246],
       [8148.96125467, 6652.67015243],
       [8198.16822747, 6671.38060947],
       [8229.87954194, 6725.54976801],
       [8153.43742976, 6783.68014746],
       [8079.83687836, 6832.59923901],
       [8065.69773499, 6881.70322097],
       [8002.96956383, 6939.52462793],
       [7964.69039281, 6999.8442929 ],
       [7912.21665689, 7039.14013322]])

In [3]:
%%time
structure = 'VCA'
margin = 500/0.46
m = 11104
n = 17216
left_max, right_max, up_max, down_max = n, 0, m, 0
for section in valid_sections:
    try:
        polygon = contours[(contours['name']==structure) & (contours['section']==section)]['vertices'].to_list()[0]
        [left, right, up, down] = [int(max(min(polygon[:, 0]) - margin, 0)),
                                   int(min(np.ceil(max(polygon[:, 0]) + margin), n)),
                                   int(max(min(polygon[:, 1]) - margin, 0)),
                                   int(min(np.ceil(max(polygon[:, 1]) + margin), m))]
        if left<left_max:
            left_max = left
        if right>right_max:
            right_max = right
        if up<up_max:
            up_max = up
        if down>down_max:
            down_max = down
    except:
        continue

CPU times: user 369 ms, sys: 12.4 ms, total: 381 ms
Wall time: 375 ms


In [23]:
left_max, right_max, up_max, down_max

(5205, 10467, 4589, 9195)

In [7]:
x.shape

(4, 265, 4606, 5262)

In [8]:
section = valid_sections[0]
src = glob(source_dir+'CSHL_cells_mark/'+stack+'/'+structure+'/'+ str(section) + '*.png')[0]
img = cv2.imread(src,-1)

In [10]:
img = np.moveaxis(img, -1, 0)
img.shape

(4, 11104, 17216)

In [4]:
%%time
structure = 'VCA'
mark = np.zeros((4,len(valid_sections), down_max-up_max, right_max-left_max), dtype=np.uint8)
for i in range(len(valid_sections)):
    section = valid_sections[i]
    src = glob(source_dir+'CSHL_cells_mark/'+stack+'/'+structure+'/'+ str(section) + '*.png')[0]
    img = cv2.imread(src,-1)
    img = np.moveaxis(img, -1, 0)
    mark[:,i,:,:] = img[:, up_max:down_max, left_max:right_max]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
CPU times: user 9min, sys: 2min 28s, total: 11min 

In [5]:
mark.shape

(4, 265, 4606, 5262)

In [24]:
viewer = neuroglancer.Viewer()

# a = np.zeros((3, 100, 100, 100), dtype=np.uint8)
# ix, iy, iz = np.meshgrid(* [np.linspace(0, 1, n) for n in a.shape[1:]], indexing='ij')
# a[0, :, :, :] = np.abs(np.sin(4 * (ix + iy))) * 255
# a[1, :, :, :] = np.abs(np.sin(4 * (iy + iz))) * 255
# a[2, :, :, :] = np.abs(np.sin(4 * (ix + iz))) * 255

a = np.zeros((100,100,100), dtype=np.uint8)
a[:,:,:]=2

b = np.zeros((100,100,100), dtype=np.uint8)
b[:,:,:]=3


with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(
        # source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/image',
        source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD594_fullres',
    )
    s.layers['VCA'] = neuroglancer.SegmentationLayer(
        source=neuroglancer.LocalVolume(mark, voxel_size=[460, 460, 20000], offset=[up_max, left_max, 94])
    )
#     s.layers['cube_2'] = neuroglancer.SegmentationLayer(
#         source=neuroglancer.LocalVolume(b, voxel_size=[10000, 10000, 10000], voxel_offset=[200, 200, 0])
#     )
#     s.voxel_coordinates = [3000, 3000, 3000]
print(viewer.state)
print(viewer)

ViewerState({"layers": [{"type": "image", "source": "precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD594_fullres", "name": "image"}, {"type": "segmentation", "source": "python://f6fe4edebb7cbe31d60aca77484d995fbd5e82e3.18a94c12a0d063476c3e6c74444649af5669482b", "name": "VCA"}]})
http://127.0.0.1:54136/v/f6fe4edebb7cbe31d60aca77484d995fbd5e82e3/


Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/sit

In [5]:
neuroglancer.ImageLayer()

ImageLayer({"type": "image"})